# Linkedin+Notion : Update metrics in content calendar from posts

## Input

### Import libraries

In [1]:
import naas
from naas_drivers import linkedin, notion, emailbuilder, gsheet
from datetime import datetime
import pandas as pd
import plotly.express as px
import os

### Variables

In [2]:
# Email
email = "jeremy.ravenel@cashstory.com"

# Lindekin cookies
LI_AT = naas.secret.get("JRV_LI_AT")
JSESSIONID = naas.secret.get("JRV_JSESSIONID")

# Linkedin profile url
profile_url = "https://www.linkedin.com/in/j%C3%A9r%C3%A9my-ravenel-8a396910"

# Notion parameters
notion_token = naas.secret.get("notion_token")
notion_database = 'https://www.notion.so/naas-official/0f806d1755b14b6d87ec8aa3bb49706b?v=1873352d6f0d4857a9976fef3beb8595'

### Get content calendar from Notion

In [3]:
db_notion = notion.connect(notion_token).database.get(notion_database)
df_notion = db_notion.df()
print("📊 Post in Notion database:", len(df_notion))

📊 Post in Notion database: 214


## Model

### Get last post in Notion database

In [4]:
df_last_post = df_notion.sort_values(by="Publication Date", ascending=False).reset_index(drop=True)
post_url = df_last_post.loc[0, "Content URL"]
print("👉 Last post url:", post_url)
# df_last_post.head(1)

👉 Last post url: https://www.linkedin.com/feed/update/urn:li:activity:6891102665217077248


### Get posts not created in notion database

In [5]:
df_posts_feed = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(profile_url, until={"POST_URL": post_url})
df_posts_feed = df_posts_feed[df_posts_feed.POST_URL != post_url]
print("🚀 New posts:", len(df_posts_feed))
# df_posts_feed

🚀 New posts: 0


### Get stats from last 50 posts

In [6]:
df_posts_stats = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(profile_url, count=50)
print("🔄 Posts stats to update:", len(df_posts_stats))
# df_posts_stats

🔄 Posts stats to update: 50


## Output

In [7]:
def create_polls_graph(uid, title, data):
    # Create dataframe
    df = pd.melt(pd.DataFrame([data]))
    df = df.sort_values(by="value")
    voters = df.value.sum()
    
    # Create fig
    fig = px.bar(df,
                 y="variable",
                 x="value",
                 orientation='h',
                 title=f"{title}<br><span style='font-size: 13px;'>Total amount of votes: {voters}</span>",
                 text="value",
                 labels={"variable": "Poll options",
                         "value": "Poll results"}
                 )
    fig.update_traces(marker_color='black')
    fig.update_layout(
        plot_bgcolor="#ffffff",
        width=600,
        height=400,
        font=dict(family="Arial", size=14, color="black"),
        paper_bgcolor="white",
        xaxis_title=None,
        xaxis_showticklabels=False,
        yaxis_title=None,
        margin_pad=10,
    )
    fig.write_html(f"{uid}.html")
    asset = naas.asset.add(f"{uid}.html", params={"inline": True})
    return asset

In [14]:
def update_dynamic_properties(page, row):
    # Page properties : dynamic
    page.number("Engagment score", float(row.ENGAGEMENT_SCORE))
    page.number("Views", int(row.VIEWS))
    page.number("Likes", int(row.LIKES))
    page.number("Comments", int(row.COMMENTS))
    page.number("Shares", int(row.SHARES))

    # Add polls
    poll_urn = row.POLL_URN
    poll_question = row.POLL_QUESTION
    poll_results = row.POLL_RESULTS
    if poll_urn and poll_question and poll_results:
        poll_graph = create_polls_graph(poll_urn, poll_question, poll_results)
        page.embed(poll_graph)
    page.update()
    print(f"✅ Post stats updated in notion for page '{row.TITLE}'.")
    return page

def update_content_notion(df, database_url):
    # Decode database id
    database_id = database_url.split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.database.query(database_id, query={})
    
    # Manage dataframe empty
    if len(df) == 0:
        print(f"🛑 Notion to update in Notion.")
        return
    
    # Loop in data
    df.COMPANY_MENTION = df.COMPANY_MENTION.fillna("")
    df.PROFILE_MENTION = df.PROFILE_MENTION.fillna("")
    for i, row in df.iterrows():
        title = row.TITLE
        post_url = row.POST_URL
        image_url = row.IMAGE_URL
        cover = None
        if image_url is not None:
            cover = image_url
        
        # Create or update page
        page_new = True
        for page in pages:
            page_temp = page.df()
            page_id = page_temp.loc[page_temp.Name == "Content URL", "Value"].values
            if page_id == post_url:
                page_new = False
                break
        try:
            if page_new:
                # Create new page in notion
                page = notion.Page.new(database_id=database_id).create()

                # Page properties : static
                page.date("Publication Date", row.PUBLISHED_DATE)
                page.title("Name", title)
                page.select("Content type", row.CONTENT)
                page.select("Platform", "LinkedIn")
                page.select("Status", "Published ✨")
                page.rich_text("Profile mention", row.PROFILE_MENTION)
                page.rich_text("Company mention", row.COMPANY_MENTION)
                page.number("Nb tags", int(row.TAGS_COUNT))
                tags = row.TAGS
                if tags is None:
                    tags = ""
                page.rich_text("Tags", tags)
                page.number("Nb emojis", int(row.EMOJIS_COUNT))
                emojis = row.EMOJIS
                if emojis is None:
                    emojis = ""
                page.rich_text("Emojis", emojis)
                page.number("Nb links", int(row.LINKS_COUNT))
                page.link("Links", row.LINKS)
                page.number("Nb characters", int(row.CHARACTER_COUNT))
                page.link("Content URL", post_url)
                
                # Page blocks
                text = row.TEXT
                split_text = text.split("\n")
                for t in split_text:
                    page.paragraph(t)

                # Add post content info
                content_title = row.CONTENT_TITLE
                if content_title is not None:
                    page.paragraph("\n")
                    page.paragraph("Content title:")
                    page.paragraph(content_title)
                content_url = row.CONTENT_URL
                if content_url is not None:
                    page.paragraph("Content url:")
                    page.paragraph(content_url)
                page.update()
                print(f"✅ Page '{title}' created in Notion.")
                
                # Page properties : dynamic
                update_dynamic_properties(page, row)
            else:
                # Page properties : dynamic
                update_dynamic_properties(page, row)            
        except Exception as e:
            print(f"❌ Error creating page '{title}' in Notion", e)
            print(row)

### Create new post page in Notion

In [15]:
update_content_notion(df_posts_feed, notion_database)

🛑 Notion to update in Notion.


,ACTIVITY_ID,PAGINATION_TOKEN,PUBLISHED_DATE,AUTHOR_NAME,SUBDESCRIPTION,TITLE,TEXT,CHARACTER_COUNT,TAGS,TAGS_COUNT,...,POLL_URN,POLL_QUESTION,POLL_RESULTS,POST_URL,VIEWS,COMMENTS,LIKES,SHARES,ENGAGEMENT_SCORE,DATE_EXTRACT


### Update post stats in Notion

In [10]:
update_content_notion(df_posts_stats, notion_database)

✅ Post stats updated in notion for page 'I’m taking a break!'.
✅ Post stats updated in notion for page '🚨 #Web3: the new buzzword of the decade? '.
✅ Post stats updated in notion for page 'May 2022 be the year we are coming together as ONE, to take actions on these 5 global issues👇'.
✅ Post stats updated in notion for page 'Everything in life is a vibration, with ups and downs. '.
✅ Post stats updated in notion for page '❓ How to learn anything?'.
✅ Post stats updated in notion for page 'Les trésoriers d’entreprise ne sont pas formés à la data… et c’est un problème majeur qui génère:'.
✅ Post stats updated in notion for page '😆😅 Humor is a serious business. '.
✅ Post stats updated in notion for page 'A simple : happy new year to you and your loved ones. '.
✅ Post stats updated in notion for page ' naas.scheduler.add(cron="*/15 * * * *") '.
✅ Post stats updated in notion for page '🎥 Video content production is a big pain in the a** 🧅 '.
✅ Post stats updated in notion for page 'Working w

<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete
✅ Post stats updated in notion for page 'How do you feel this second week of 2022?'.
✅ Post stats updated in notion for page '🚀 Finally, Naas Open Source Progam is ready! '.
✅ Post stats updated in notion for page '🐝 💥 There is a lot of buzzwords in the data space but one is highly under-rated: Citizen Data Scientist (CDS). '.
✅ Post stats updated in notion for page 'I'm BACK 😊, feeling more motivated than ever to change the game in the data space! '.
👌 Well done! Your Assets has been sent to production.



...

PS: to remove the "Assets" feature, just replace .add by .delete
✅ Post stats updated in notion for page 'How do you feel this 1st week of 2022? '.
✅ Post stats updated in notion for page '🚀✨ Notebooks + Low-code libraries are opening doors to new horizons, where there is no more bridge to build between tech and non-tech world. '.
✅ Post stats updated in notion for page '⚡️🍼 Having kids does not have to take all your productive time.'.
✅ Post stats updated in notion for page 'I’m having hard times posting regularly those days. 🤯 '.
✅ Post stats updated in notion for page '☝️ My father used to say « Stop setting up new goals all the time, finish what you started first. »'.
✅ Post stats updated in notion for page '81 🎉 That's the number of contributors that applied to the  naas.ai Open Source Contributor program we are going to kickstart on Tuesday! 🌎⚡️'.
✅ Post stats updated in notion for page 'There is days like this where all the planets align… 🪐'.
✅ Post stats updated in notion for p

<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete
✅ Post stats updated in notion for page 'How do you feel this week 3 of 2022? '.
✅ Post stats updated in notion for page '🛑 Open Source is not a business model. '.
✅ Post stats updated in notion for page 'What motivates you the most in the AI field?'.
✅ Post stats updated in notion for page 'There are no problems in life.'.
✅ Post stats updated in notion for page '"In community we trust" 🙏 '.
✅ Post stats updated in notion for page '🌍💰Contribute to opensource and make money from the expertise you build on the product! '.
✅ Post stats updated in notion for page 'https://lnkd.in/e_ti8TDC'.


### Archive data in naas

In [16]:
def archive_csv(df, filename="LINKEDIN_POSTS_FEED.csv", output_dir=None):
    # Add timestamp to df
    df["DATE_EXTRACT"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Get archive
    df_archive = pd.DataFrame()
    files = os.listdir(output_dir)
    for f in files:
        if f == filename:
            df_archive = pd.read_csv(filename)
            break
            
    # Concat
    df = pd.concat([df, df_archive], axis=0).drop_duplicates("POST_URN", keep="first")
    df.to_csv(filename, index=False)
    return df

In [11]:
df = archive_csv(df_posts_feed)
print("💾 Linkedin posts archives:", len(df))

NameError: name 'os' is not defined